In [25]:
%load_ext autoreload
%autoreload 2
from config.config import *
import numpy as np
import roboticstoolbox as rtb
import spatialmath as sm
import urchin
from urchin import URDF

import os


import swift


print(urchin.__version__)

urdf = URDF.load(os.path.join(os.getcwd(), PANDA_URDF))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.0.27


ValueError: string is not a file: `/home/vahid/sbu/Screw-Panda/models/franka_description/robots/package://franka_description/meshes/visual/link0.dae`